<a href="https://colab.research.google.com/github/sou-venir/Dream-nori/blob/main/%EC%BD%94%EB%9E%A9%EC%9D%84%EC%9C%84%ED%95%9C%EB%93%9C%EB%A6%BC%EB%86%80%EC%9D%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# [1] 필수 라이브러리 설치 (코랩 환경 전용)
!pip install flask-socketio flask-ngrok pyngrok openai python-socketio
!fuser -k 5000/tcp # 기존 포트 정리

import os
import json
from flask import Flask, render_template_string
from flask_socketio import SocketIO, emit
from pyngrok import ngrok
from google.colab import userdata
import openai
from google.colab import drive
import google.generativeai as genai

# [2] 구글 드라이브 마운트
drive.mount('/content/drive')
SAVE_PATH = '/content/drive/MyDrive/ChatData'
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

# [3] 보안 및 AI 설정
gemini_model = None # 전역 변수 초기화 (에러 방지)

try:
    NGROK_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    client = openai.OpenAI(api_key=OPENAI_API_KEY)

    # 구글 키는 없을 수도 있으니 예외처리
    try:
        GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
        if GOOGLE_API_KEY:
            genai.configure(api_key=GOOGLE_API_KEY)
            gemini_model = genai.GenerativeModel('gemini-1.5-pro-latest')
    except:
        print("⚠️ 구글 API 키가 설정되지 않았습니다. Gemini 모델은 사용할 수 없습니다.")

    ngrok.set_auth_token(NGROK_TOKEN)
except Exception as e:
    print(f"❌ 설정 확인 필요: {e}")

app = Flask(__name__)
socketio = SocketIO(app, cors_allowed_origins="*")

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE, theme=state.get('theme'))

DATA_FILE = os.path.join(SAVE_PATH, "save_data.json")

def save_data():
    with open(DATA_FILE, "w", encoding="utf-8") as f:
        json.dump(state, f, ensure_ascii=False, indent=4)

def load_data():
    if os.path.exists(DATA_FILE):
        try:
            with open(DATA_FILE, "r", encoding="utf-8") as f:
                return json.load(f)
        except:
            return None
    return None

# [4] 서버 상태 관리
initial_state = {
    "session_title": "드림놀이",
    "theme": {"bg": "#ffffff", "panel": "#f1f3f5", "accent": "#e91e63"},
    "ai_model": "gpt-5.2",
    "admin_password": "3896",
    "is_locked": False,
    "inputs": {"user1": None, "user2": None},
    "profiles": {
        "user1": {"name": "Player 1", "bio": "", "canon": ""},
        "user2": {"name": "Player 2", "bio": "", "canon": ""}
    },
    "history": [],
    "ai_history": [],
    "summary": "",
    "prologue": "",
    "sys_prompt": "당신은 숙련된 TRPG 마스터입니다.",
    "lorebook": [],
    "examples": [{"q":"", "a":""}, {"q":"", "a":""}, {"q":"", "a":""}]
}

saved_state = load_data()
state = saved_state if saved_state else initial_state

# [5] 테마 분석 로직
def analyze_theme_color(title, sys_prompt):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{
                "role": "system",
                "content": "너는 웹 디자인 전문가야. JSON으로 답해: {\"bg\": \"색\", \"panel\": \"색\", \"accent\": \"색\"}"
            }, {
                "role": "user",
                "content": f"제목: {title}\n설정: {sys_prompt}"
            }],
            response_format={ "type": "json_object" }
        )
        palette = json.loads(response.choices[0].message.content)
        default = {"bg": "#ffffff", "panel": "#f1f3f5", "accent": "#e91e63"}
        default.update(palette)
        return default
    except:
        return {"bg": "#ffffff", "panel": "#f1f3f5", "accent": "#e91e63"}

# [6] HTML 템플릿 (메인 UI 보존, 모달만 수정)
HTML_TEMPLATE = """<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>드림놀이</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.0.1/socket.io.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
    <style>
        :root {
            --bg: {{ theme.bg if theme else '#ffffff' }};
            --panel: {{ theme.panel if theme else '#f1f3f5' }};
            --accent: {{ theme.accent if theme else '#e91e63' }};
        }
        /* [메인 UI 스타일 유지] */
        html, body { height: 100%; margin: 0; overflow: hidden; }
        body { font-family: 'Pretendard', sans-serif; display: flex; background: var(--bg); color: #000; }
        div, p, span, h1, h2, h3, h4, input, textarea, select, button { color: #000 !important; }

        #main { flex: 1; display: flex; flex-direction: column; height: 100vh; border-right: 1px solid rgba(0,0,0,0.05); }
        #chat-window { flex: 1; overflow-y: auto; padding: 30px 10%; display: flex; flex-direction: column; gap: 15px; }
        #sidebar { width: 320px; height: 100vh; background: var(--panel); padding: 20px; overflow-y: auto; display: flex; flex-direction: column; gap: 12px; }

        textarea, input, select {
            background: var(--bg) !important;
            border: 1px solid rgba(0, 0, 0, 0.1) !important;
            border-radius: 10px; padding: 10px; width: 100%; box-sizing: border-box;
            resize: none !important;
        }
        #msg-input { background: var(--panel) !important; height: 80px; border: 1px solid rgba(0,0,0,0.15) !important; }
        textarea:focus, input:focus { outline: none; border-color: var(--accent) !important; }

        .bubble { padding: 15px 20px; border-radius: 15px; max-width: 85%; line-height: 1.6; white-space: pre-wrap; background: rgba(0,0,0,0.03); }
        .center-ai { align-self: center; background: var(--panel) !important; border-left: 5px solid var(--accent); width: 100%; max-width: 800px; box-shadow: 0 4px 15px rgba(0,0,0,0.05); }
        .user-bubble { align-self: flex-end; border-right: 5px solid var(--accent); background: var(--bg); }

        button { cursor: pointer; border: none; border-radius: 8px; background: var(--accent); padding: 10px; font-weight: bold; color:white !important; }
        button:hover { opacity: 0.8; }
        .btn-reset { background: #ff4444 !important; color: #ffffff !important; margin-top: 20px; }

        /* [모달 스타일 수정됨] */
        #admin-modal { display: none; position: fixed; z-index: 10000; left: 0; top: 0; width: 100vw; height: 100vh; background: rgba(0,0,0,0.6); backdrop-filter: blur(5px); align-items: center; justify-content: center; }
        .modal-content { width: 95%; max-width: 1200px; height: 85vh; background: #fff; border-radius: 16px; display: flex; flex-direction: column; overflow: hidden; box-shadow: 0 20px 60px rgba(0,0,0,0.3); }
        .modal-header { height: 60px; display: flex; justify-content: space-between; align-items: center; padding: 0 25px; background: #f8f9fa; border-bottom: 1px solid #eee; }

        .tab-btn { border: none; background: none; padding: 0 15px; font-size: 14px; font-weight: 600; color: #777; cursor: pointer; height: 100%; position: relative; }
        .tab-btn.active { color: var(--accent); }
        .tab-btn.active::after { content: ""; position: absolute; bottom: 0; left: 0; width: 100%; height: 3px; background: var(--accent); }

        .modal-body { flex: 1; display: flex; overflow: hidden; }
        .tab-content { display: none; width: 100%; height: 100%; flex-direction: row; }
        .tab-content.active { display: flex; }

        .editor-side { flex: 1.3; padding: 25px; display: flex; flex-direction: column; gap: 15px; overflow-y: auto; border-right: 1px solid #f0f0f0; }
        .list-side { flex: 0.7; padding: 25px; background: #fafafa; display: flex; flex-direction: column; gap: 15px; overflow-y: auto; }

        label { font-size: 12px; font-weight: 800; color: #999; margin-bottom: -5px; }
        .save-btn { background: var(--accent); color: white !important; padding: 15px; border-radius: 10px; font-weight: bold; margin-top: 5px; }

        /* 학습 데이터 블록 스타일 */
        .ex-block { background: #fff; border: 1px solid #eee; padding: 15px; border-radius: 8px; display: flex; flex-direction: column; gap: 8px; }
        .ex-block input { border: 1px solid #eee !important; font-size: 13px; }
    </style>
</head>
<body>
    <div id="main">
        <div id="chat-window"><div id="chat-content"></div></div>
        <div id="input-area" style="padding:20px; background: var(--bg);">
            <div id="status" style="font-size: 12px; margin-bottom: 5px; color: var(--accent); font-weight: bold;">대기 중</div>
            <div style="display:flex; gap:10px;">
                <textarea id="msg-input" placeholder="설정 완료 후 잠금 버튼을 눌러주세요."></textarea>
                <button onclick="send()" style="width:80px;">전송</button>
            </div>
        </div>
    </div>

   <div id="sidebar">
        <h3>🎭 설정</h3>
        <select id="user-role" onchange="refreshUI()">
            <option value="user1">Player 1</option>
            <option value="user2">Player 2</option>
        </select>
        <input type="text" id="p-name" placeholder="이름">
        <textarea id="p-bio" style="height:120px;" placeholder="캐릭터 설정"></textarea>
        <textarea id="p-canon" style="height:80px;" placeholder="관계 설정"></textarea>
        <button onclick="saveProfile()" id="ready-btn">✅ 설정 저장 및 준비 완료</button>
        <div id="ready-status" style="font-size:11px; margin-top:5px; color:#666;">대기 중...</div>
        <div style="flex: 1;"></div>
        <button onclick="requestAdmin()" style="background:transparent; color:#999 !important; border: 1px solid #ddd;">⚙️ 마스터 설정 </button>
    </div>

    <!-- 모달 창 -->
    <div id="admin-modal">
        <div class="modal-content">
            <div class="modal-header">
                <div class="tab-group">
                    <button class="tab-btn active" onclick="openTab(event, 't-base')">⚙️ 엔진</button>
                    <button class="tab-btn" onclick="openTab(event, 't-story')">🎬 서사</button>
                    <button class="tab-btn" onclick="openTab(event, 't-ex')">💡 학습</button>
                    <button class="tab-btn" onclick="openTab(event, 't-lore')">📚 키워드</button>
                </div>
                <button onclick="closeModal()" style="width:32px; height:32px; border-radius:50%; background:#eee; color:black !important;">✕</button>
            </div>

            <div class="modal-body">
                <!-- 1. 엔진 -->
                <div id="t-base" class="tab-content active">
                    <div class="editor-side">
                        <label>AI 모델 선택</label>
                        <select id="m-ai-model">
                            <option value="gpt-5.2">OpenAI GPT-5.2</option>
                            <option value="gpt-4o">OpenAI GPT-4o</option>
                            <option value="gemini-1.5-pro-latest">Google Gemini 1.5 Pro</option>
                        </select>
                        <label>시스템 프롬프트 (AI 지침)</label>
                        <textarea id="m-sys" placeholder="AI에게 줄 지침..."></textarea>
                        <button onclick="saveMaster()" class="save-btn">💾 엔진 설정 저장</button>
                    </div>
                    <div class="list-side">
                        <label>안내</label>
                        <p style="font-size:13px; color:#666;">엔진 모델과 전체적인 AI의 페르소나를 결정합니다.</p>
                        <button class="btn-reset" onclick="sessionReset()" style="margin-top: auto;">⚠️ 세션 완전 초기화</button>
                    </div>
                </div>

                <!-- 2. 서사 (요약 이동됨) -->
                <div id="t-story" class="tab-content">
                    <div class="editor-side">
                        <label>🏷️ 세션 제목</label>
                        <input type="text" id="m-title" placeholder="제목">
                        <label>📖 프롤로그</label>
                        <textarea id="m-pro" placeholder="이야기의 시작..."></textarea>
                        <button onclick="saveMaster()" class="save-btn">💾 모든 서사 저장</button>
                    </div>
                    <div class="list-side">
                        <label>📌 현재 상황 요약 (줄거리)</label>
                        <textarea id="m-sum" style="height:200px;" placeholder="지금까지의 핵심 내용... (AI가 이걸 기억합니다)"></textarea>
                        <label>💡 서사 팁</label>
                        <p style="font-size:13px; color:#666;">오른쪽(여기) 내용은 AI가 이야기 맥락을 파악할 때 가장 중요하게 참고해.</p>
                    </div>
                </div>

                <!-- 3. 학습 (3쌍 분리됨) -->
                <div id="t-ex" class="tab-content">
                    <div class="editor-side">
                        <label>💡 말투 학습 (예시 대화)</label>

                        <div class="ex-block">
                            <label>Example 1</label>
                            <input type="text" id="ex-q-0" placeholder="[유저]: 안녕?">
                            <input type="text" id="ex-a-0" placeholder="[AI]: 안녕하세요! 무엇을 도와드릴까요?">
                        </div>
                        <div class="ex-block">
                            <label>Example 2</label>
                            <input type="text" id="ex-q-1" placeholder="[유저]: 너는 누구야?">
                            <input type="text" id="ex-a-1" placeholder="[AI]: 저는 당신의 모험을 돕는 가이드입니다.">
                        </div>
                        <div class="ex-block">
                            <label>Example 3</label>
                            <input type="text" id="ex-q-2" placeholder="[유저]: 상황 설명 좀">
                            <input type="text" id="ex-a-2" placeholder="[AI]: 현재 당신은 어두운 숲 속에 서 있습니다.">
                        </div>
                        <button onclick="saveExamples()" class="save-btn">💡 학습 데이터 저장</button>
                    </div>
                    <div class="list-side">
                        <label>도움말</label>
                        <p style="font-size:13px; color:#666;">AI가 따라했으면 하는 말투를 직접 입력해줘.<br>왼쪽 칸들을 채우면 AI가 "아, 이렇게 말하면 되는구나!" 하고 배워.</p>
                    </div>
                </div>

                <!-- 4. 키워드 (가로로 넓어짐) -->
                <div id="t-lore" class="tab-content">
                    <div class="editor-side">
                        <label>🔍 키워드 이름</label>
                        <input type="text" id="kw-t" placeholder="이름 (예: 엘프)">

                        <label>🎯 트리거 (콤마로 구분)</label>
                        <!-- 스타일 width 100% 적용 -->
                        <input type="text" id="tag-input" placeholder="엘프, 요정, 숲의주인" style="width: 100% !important;">

                        <label>📝 상세 설정</label>
                        <textarea id="kw-c" placeholder="AI에게 전달할 설정 내용..."></textarea>
                        <button onclick="addLoreWithTags()" class="save-btn">➕ 키워드 저장</button>
                    </div>
                    <div class="list-side">
                        <label>📋 저장된 키워드</label>
                        <div id="lore-list" style="flex: 1; overflow-y: auto; display:flex; flex-direction:column; gap:8px;"></div>
                    </div>
                </div>
            </div>
        </div>
    </div>

<script>
    const socket = io();
    let gState = null;

    // 상태 업데이트
    socket.on('status_update', d => {
        const statusEl = document.getElementById('status');
        if(statusEl) {
            statusEl.innerText = d.msg;
            statusEl.style.color = d.msg.includes('❌') ? 'red' : 'var(--accent)';
        }
    });

    // 초기 데이터 로드
    socket.on('initial_state', data => {
        gState = data;
        if (data.theme) {
            const root = document.documentElement.style;
            root.setProperty('--bg', data.theme.bg);
            root.setProperty('--panel', data.theme.panel);
            root.setProperty('--accent', data.theme.accent);
        }
        refreshUI();
    });

    // 타이핑 효과
    function typeWriter(element, text, i = 0) {
        if (i === 0) { element.innerHTML = ""; element.style.whiteSpace = "pre-wrap"; }
        if (i < text.length) {
            element.textContent += text.charAt(i);
            i++;
            const win = document.getElementById('chat-window');
            win.scrollTop = win.scrollHeight;
            setTimeout(() => typeWriter(element, text, i), 35);
        } else {
            element.innerHTML = marked.parse(text);
        }
    }

    // UI 새로고침
    function refreshUI() {
        if(!gState) return;
        renderChat();
        renderLore();
        applyLockUI();

        const activeId = document.activeElement.id;
        const role = document.getElementById('user-role').value;
        const p = gState.profiles[role];

        // 프로필
        if(activeId !== 'p-name') document.getElementById('p-name').value = p.name || "";
        if(activeId !== 'p-bio') document.getElementById('p-bio').value = p.bio || "";
        if(activeId !== 'p-canon') document.getElementById('p-canon').value = p.canon || "";

        // 마스터 설정
        if(activeId !== 'm-title') document.getElementById('m-title').value = gState.session_title || "";
        if(activeId !== 'm-sys') document.getElementById('m-sys').value = gState.sys_prompt || "";
        if(activeId !== 'm-pro') document.getElementById('m-pro').value = gState.prologue || "";
        if(activeId !== 'm-sum') document.getElementById('m-sum').value = gState.summary || "";

        // 학습 데이터 (3쌍)
        if(gState.examples) {
            for(let i=0; i<3; i++) {
                if(gState.examples[i]) {
                    if(activeId !== `ex-q-${i}`) document.getElementById(`ex-q-${i}`).value = gState.examples[i].q || "";
                    if(activeId !== `ex-a-${i}`) document.getElementById(`ex-a-${i}`).value = gState.examples[i].a || "";
                }
            }
        }
    }

    // 설정 잠금
    function applyLockUI() {
        if(!gState) return;
        const role = document.getElementById('user-role').value;
        const p = gState.profiles[role];
        const locked = (p.name && p.name !== "Player 1" && p.name !== "Player 2");

        document.getElementById('p-name').readOnly = locked;
        document.getElementById('p-bio').readOnly = locked;
        document.getElementById('p-canon').readOnly = locked;
        document.getElementById('ready-btn').disabled = locked;
        document.getElementById('ready-btn').innerText = locked ? "🔒 설정 고정됨" : "✅ 설정 저장 및 준비 완료";
    }

    // 채팅 렌더링
    function renderChat() {
        let h = `<div style="text-align:center; padding:20px; color:var(--accent); font-weight:bold; font-size:1.4em;">${gState.session_title}</div>`;
        h += `<div class="bubble center-ai"><b>[PROLOGUE]</b><br>${marked.parse(gState.prologue || "")}</div>`;

        const contentDiv = document.getElementById('chat-content');
        const history = gState.ai_history;
        const role = document.getElementById('user-role').value;
        const pName = gState.profiles[role].name;

        history.forEach((msg, index) => {
            const isLastMsg = (index === history.length - 1);
            const isAI = msg.startsWith("**AI**:");
            const isUser = pName && msg.includes(`**${pName}**:`);

            if (isLastMsg && isAI) {
                const bubbleId = `typing-${index}`;
                h += `<div id="${bubbleId}" class="bubble center-ai"></div>`;
                contentDiv.innerHTML = h;
                const targetElement = document.getElementById(bubbleId);
                typeWriter(targetElement, msg);
            } else {
                h += `<div class="bubble ${isUser ? 'user-bubble' : 'center-ai'}">${marked.parse(msg)}</div>`;
            }
        });
        if (history.length === 0 || !history[history.length-1].startsWith("**AI**:")) {
            contentDiv.innerHTML = h;
        }
        const win = document.getElementById('chat-window');
        win.scrollTop = win.scrollHeight;
    }

    function send() {
        const input = document.getElementById('msg-input');
        const text = input.value.trim();
        if(!text) return;
        socket.emit('client_message', { uid: document.getElementById('user-role').value, text });
        input.value = '';
    }

    // 모달 및 관리자 기능
    function requestAdmin() {
        const pw = prompt("관리자 비밀번호:");
        if(pw) socket.emit('check_admin', { password: pw });
    }
    socket.on('admin_auth_res', d => {
        if(d.success) {
            document.getElementById('admin-modal').style.display = 'flex';
            openTab(null, 't-base');
            refreshUI();
        } else {
            alert("비밀번호 불일치");
        }
    });

    function saveMaster() {
        const masterData = {
            title: document.getElementById('m-title').value,
            sys: document.getElementById('m-sys').value,
            pro: document.getElementById('m-pro').value,
            sum: document.getElementById('m-sum').value,
            model: document.getElementById('m-ai-model').value
        };
        socket.emit('save_master_base', masterData);
        alert("저장되었습니다!");
    }

    function saveExamples() {
        const exs = [];
        for(let i=0; i<3; i++) {
            exs.push({
                q: document.getElementById(`ex-q-${i}`).value,
                a: document.getElementById(`ex-a-${i}`).value
            });
        }
        socket.emit('save_examples', exs);
        alert("학습 데이터 저장 완료.");
    }

    // [키워드 추가 함수 - ID 수정 및 공백 처리 강화]
    function addLoreWithTags() {
        const title = document.getElementById('kw-t').value;
        const tags = document.getElementById('tag-input').value;
        const content = document.getElementById('kw-c').value;

        if(!title) return alert("키워드 이름을 입력해줘!");
        if(!content) return alert("설정 내용을 입력해줘!");

        socket.emit('add_lore', {
            title: title,
            triggers: tags,
            content: content,
            priority: 0
        });

        document.getElementById('kw-t').value = "";
        document.getElementById('tag-input').value = "";
        document.getElementById('kw-c').value = "";
    }

    function renderLore() {
        const listDiv = document.getElementById('lore-list');
        if(!gState || !gState.lorebook) return;
        listDiv.innerHTML = gState.lorebook.map((l, i) => `
            <div style="padding:8px; background:rgba(0,0,0,0.03); margin-bottom:5px; border-radius:8px; display:flex; justify-content:space-between; align-items:center;">
                <span onclick="editLore(${i})" style="cursor:pointer; flex:1; font-size:13px;">
                    <b>${l.title}</b> <small style="color:#666;">[${l.triggers}]</small>
                </span>
                <button onclick="socket.emit('del_lore', {index:${i}})" style="padding:2px 8px; font-size:11px; background:#ff4444;">삭제</button>
            </div>`).join('');
    }

    function editLore(idx) {
        const l = gState.lorebook[idx];
        document.getElementById('kw-t').value = l.title;
        document.getElementById('tag-input').value = l.triggers || "";
        document.getElementById('kw-c').value = l.content;
        if(confirm("수정 모드: 목록에서 삭제하고 입력창으로 가져올까?")) {
            socket.emit('del_lore', { index: idx });
        }
    }

    function openTab(evt, id) {
        document.querySelectorAll('.tab-content').forEach(c => c.classList.remove('active'));
        document.querySelectorAll('.tab-btn').forEach(b => b.classList.remove('active'));
        document.getElementById(id).classList.add('active');
        if(evt) evt.currentTarget.classList.add('active');
        else document.querySelector(`[onclick="openTab(event, '${id}')"]`).classList.add('active');
    }
    function closeModal() { document.getElementById('admin-modal').style.display='none'; }

    function saveProfile() {
        const role = document.getElementById('user-role').value;
        const name = document.getElementById('p-name').value;
        if(!name || name.includes("Player")) return alert("이름을 입력해주세요!");
        if(confirm("이 설정으로 확정할까? (수정 불가)")) {
            socket.emit('update_profile', {
                uid: role, name: name,
                bio: document.getElementById('p-bio').value,
                canon: document.getElementById('p-canon').value
            });
        }
    }
    function sessionReset() {
        if(confirm("정말 초기화해?") && prompt("관리자 비번:") === "3896") socket.emit('reset_session', { password: "3896" });
    }

    document.getElementById('msg-input').addEventListener('keydown', e => {
        if(e.key==='Enter' && !e.shiftKey) { e.preventDefault(); send(); }
    });
    socket.emit('request_data');
</script>
</body>
</html>
"""

# [7] 서버 로직
def broadcast_state(msg=None):
    save_data()
    emit('initial_state', state, broadcast=True)
    if msg: emit('status_update', {'msg': msg}, broadcast=True)

@socketio.on('request_data')
def handle_request(): emit('initial_state', state)

@socketio.on('client_message')
def on_client_message(data):
    user_text = data.get('text', '').strip()
    uid = data.get('uid')
    if not user_text: return

    # 1. 키워드 검색 (공백 제거 후 매칭)
    active_context = []
    if state.get('lorebook'):
        for l in state['lorebook']:
            triggers = [t.strip() for t in l.get('triggers', '').split(',') if t.strip()]
            if any(t in user_text for t in triggers):
                active_context.append(f"[{l['title']} 설정]: {l['content']}")

    # 2. 메시지 구성
    messages = [{"role": "system", "content": f"{state['sys_prompt']}\n\n[현재 줄거리 요약]: {state['summary']}\n[관련 키워드 정보]: {' '.join(active_context[:3])}"}]

    # 3. 학습 데이터 (3쌍)
    if state.get('examples'):
        for ex in state['examples']:
            if ex.get('q') and ex.get('a'):
                messages.append({"role": "user", "content": ex['q']})
                messages.append({"role": "assistant", "content": ex['a']})

    # 4. 최근 대화
    for h in state['ai_history'][-15:]:
        messages.append({"role": "assistant" if h.startswith("**AI**") else "user", "content": h})

    current_name = state['profiles'].get(uid, {}).get('name', 'User')
    messages.append({"role": "user", "content": f"{current_name}: {user_text}"})

    ai_response = ""
    try:
        current_model = state.get("ai_model", "gpt-5.2")
        emit('status_update', {'msg': f'🤔 {current_model} 생각 중...'}, broadcast=True)

        if "gemini" in current_model.lower():
            if gemini_model is None:
                raise Exception("구글 API 키가 없어 Gemini를 쓸 수 없어!")

            full_prompt = f"{state['sys_prompt']}\n[줄거리]: {state['summary']}\n"
            full_prompt += "\n".join(state['ai_history'][-10:])
            full_prompt += f"\n{current_name}: {user_text}\nAI:"

            try:
                # 사용자가 입력한 모델명 시도, 실패 시 기본값
                gen_model = genai.GenerativeModel(current_model)
                response = gen_model.generate_content(full_prompt)
            except:
                response = gemini_model.generate_content(full_prompt)
            ai_response = response.text
        else:
            response = client.chat.completions.create(model=current_model, messages=messages)
            ai_response = response.choices[0].message.content

        state["ai_history"].append(f"**{current_name}**: {user_text}")
        state["ai_history"].append(f"**AI**: {ai_response}")
        broadcast_state('✅ 완료')

    except Exception as e:
        emit('status_update', {'msg': f'❌ 에러: {str(e)}'}, broadcast=True)

@socketio.on('save_master_base')
def on_save_master(data):
    state.update({
        "session_title": data['title'],
        "sys_prompt": data['sys'],
        "prologue": data['pro'],
        "summary": data['sum'],
        "ai_model": data.get('model', 'gpt-5.2')
    })
    try:
        new_theme = analyze_theme_color(state["session_title"], state["sys_prompt"])
        state["theme"] = new_theme
    except: pass
    broadcast_state("💾 설정 저장됨")

@socketio.on('save_examples')
def on_save_examples(data):
    state["examples"] = data
    broadcast_state("💡 학습 데이터 저장됨")

@socketio.on('update_profile')
def on_profile(data):
    # 필요한 데이터만 골라서 저장 (오염 방지)
    state["profiles"][data['uid']].update({
        "name": data['name'],
        "bio": data['bio'],
        "canon": data['canon']
    })
    broadcast_state("👤 프로필 업데이트")

@socketio.on('add_lore')
def on_add_lore(data):
    state.setdefault("lorebook", []).append(data)
    broadcast_state("➕ 키워드 추가됨")

@socketio.on('del_lore')
def on_del_lore(data):
    try:
        state["lorebook"].pop(data['index'])
        broadcast_state("🗑️ 키워드 삭제됨")
    except: pass

@socketio.on('check_admin')
def check_admin(data):
    emit('admin_auth_res', {'success': str(data.get('password')) == str(state.get('admin_password'))})

@socketio.on('reset_session')
def on_reset_session(data):
    if str(data.get('password')) == str(state.get('admin_password')):
        state.update({"ai_history": [], "lorebook": [], "summary": "", "is_locked": False})
        broadcast_state("♻️ 초기화 완료")

if __name__ == '__main__':
    try:
        import subprocess
        subprocess.run(["pkill", "-9", "ngrok"])
        from pyngrok import ngrok
        ngrok.kill()

        public_url = ngrok.connect(5000).public_url
        print(f"\n🚀 서버 시작! 접속 주소: {public_url}")
        socketio.run(app, host='0.0.0.0', port=5000, allow_unsafe_werkzeug=True)
    except Exception as e:
        print(f"❌ 실행 오류: {e}")